In [13]:
import utils.cs_vqe_tools as c_tools
import utils.qonversion_tools as qonvert
import utils.bit_tools as bit
import utils.circuit_tools as circ
import utils.linalg_tools as la
import utils.plotting_tools as plot
from utils.UCCSD_ansatz import Get_UCCSD_ia_terms, Get_UCCSD_ijab_terms, Fermi_ops_to_qubit_ops

import cs_vqe_classes.cs_vqe as c
import cs_vqe_classes.eigenstate as eig
import cs_vqe_classes.cs_vqe_circuit as cs_circ

import ast
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from copy import deepcopy
from openfermion.linalg import LinearQubitOperator, get_sparse_operator, get_ground_state, jw_hartree_fock_state
import itertools
from statistics import median

from qiskit.circuit.parameter import Parameter
from qiskit.circuit.quantumcircuit import QuantumCircuit, QuantumRegister
from qiskit import QuantumCircuit, BasicAer, execute
from qiskit.visualization import plot_histogram
from qiskit.utils import QuantumInstance
from qiskit.aqua.components.optimizers import SLSQP, COBYLA
from qiskit.algorithms import VQE
from qiskit import Aer

import cirq
from openfermion.linalg import get_sparse_operator, get_ground_state, jw_hartree_fock_state, jw_configuration_state

from qiskit.providers.aer import AerError

In [14]:
import openfermion
import openfermionpyscf
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator, uccsd_generator,
                               uccsd_convert_amplitude_format)

singlet_bool = True # Set general UCCSD or singlet UCCSD.

bond_len = 0.772#1.45#1.342396
atom_1 = 'He'
atom_2 = 'H'
#atom_3 = 'H'
basis = '3-21g'
multiplicity = 1
charge = 1

coordinate_1 = (0.0, 0.0, 0.0)
coordinate_2 = (0.0, 0.0, bond_len)
#coordinate_3 = (0.0, bond_len, 0.0)

geometry = [(atom_1, coordinate_1), (atom_2, coordinate_2)]#, (atom_3, coordinate_3)]

molecule_data = MolecularData(geometry, basis, multiplicity, charge, description='Test')
#molecule.load()

# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_cisd = 0
run_ccsd = 0
run_fci = 1
delete_input = True
delete_output = True

# Run pyscf.
molecule = run_pyscf(molecule_data,
                     run_scf=run_scf,
                     run_mp2=run_mp2,
                     run_cisd=run_cisd,
                     run_ccsd=run_ccsd,
                     run_fci=run_fci)

#molecule.load()
#print(molecule)

ham_f = get_fermion_operator(molecule.get_molecular_hamiltonian())
ham_q = jordan_wigner(ham_f)
print('Hamiltonian:', '\n', ham_q, '\n')

scf = True      # Hartree-Fock.
mp2 = True      # Moller-Plesset 2.
cisd = True     # Configuration interaction singles and doubles.
ccsd = True     # Coupled cluster singles and doubles.
fci = True      # Full configuration interaction.

calculated_molecule = run_pyscf(molecule_data, scf, mp2, cisd, ccsd, fci)

if ccsd:
    ccsd_single_amps = calculated_molecule.ccsd_single_amps
    ccsd_double_amps = calculated_molecule.ccsd_double_amps

num_electrons = calculated_molecule.n_electrons
num_qubits = 2*calculated_molecule.n_orbitals

if singlet_bool:
    # Get singlet UCCSD generator.
    packed_amps = uccsd_singlet_get_packed_amplitudes(ccsd_single_amps,  ccsd_double_amps, num_qubits, num_electrons)
    ucc_op = uccsd_singlet_generator(packed_amps, num_qubits, num_electrons)
    #print(ucc_sing)

else:
    # Get general UCCSD operator.
    ucc_op = uccsd_generator(ccsd_single_amps, ccsd_double_amps)
    #print(ucc_op)
    
ucc_q = jordan_wigner(ucc_op)
ucc = qonvert.QubitOperator_to_dict(ucc_q, num_qubits)
print('UCCSD ansatz:', '\n', ucc_q)

Hamiltonian: 
 (1.1324434021069372+0j) [] +
(-0.02581025441481341+0j) [X0 X1 Y2 Y3] +
(-0.012799934786437657+0j) [X0 X1 Y2 Z3 Z4 Y5] +
(-0.025159645839897188+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Y7] +
(-0.012799934786437657+0j) [X0 X1 X3 X4] +
(-0.025159645839897195+0j) [X0 X1 X3 Z4 Z5 X6] +
(-0.01711452882177503+0j) [X0 X1 Y4 Y5] +
(-0.0009891049103639073+0j) [X0 X1 Y4 Z5 Z6 Y7] +
(-0.0009891049103639073+0j) [X0 X1 X5 X6] +
(-0.054698565746850156+0j) [X0 X1 Y6 Y7] +
(0.02581025441481341+0j) [X0 Y1 Y2 X3] +
(0.012799934786437657+0j) [X0 Y1 Y2 Z3 Z4 X5] +
(0.025159645839897188+0j) [X0 Y1 Y2 Z3 Z4 Z5 Z6 X7] +
(-0.012799934786437657+0j) [X0 Y1 Y3 X4] +
(-0.025159645839897195+0j) [X0 Y1 Y3 Z4 Z5 X6] +
(0.01711452882177503+0j) [X0 Y1 Y4 X5] +
(0.0009891049103639073+0j) [X0 Y1 Y4 Z5 Z6 X7] +
(-0.0009891049103639073+0j) [X0 Y1 Y5 X6] +
(0.054698565746850156+0j) [X0 Y1 Y6 X7] +
(0.002577905179407553+0j) [X0 Z1 X2] +
(0.004766148105680499+0j) [X0 Z1 X2 X3 Z4 X5] +
(-0.009264702923306941+0j) [X0 Z1 X2 X3 Z

UCCSD ansatz: 
 0.006478936130102838j [X0 X1 X2 Y3] +
0.006478936130102838j [X0 X1 Y2 X3] +
0.0020562744029710704j [X0 X1 X4 Y5] +
0.0020562744029710704j [X0 X1 Y4 X5] +
0.005083283191416048j [X0 X1 X6 Y7] +
0.005083283191416048j [X0 X1 Y6 X7] +
-0.006478936130102838j [X0 Y1 X2 X3] +
0.006478936130102838j [X0 Y1 Y2 Y3] +
-0.0020562744029710704j [X0 Y1 X4 X5] +
0.0020562744029710704j [X0 Y1 Y4 Y5] +
-0.005083283191416048j [X0 Y1 X6 X7] +
0.005083283191416048j [X0 Y1 Y6 Y7] +
-0.006360390047729467j [X0 Z1 Y2] +
-0.002645411873769119j [X0 Z1 Z2 Z3 Y4] +
0.0009221551709274561j [X0 Z1 Z2 Z3 Z4 Z5 Y6] +
-0.006478936130102838j [Y0 X1 X2 X3] +
0.006478936130102838j [Y0 X1 Y2 Y3] +
-0.0020562744029710704j [Y0 X1 X4 X5] +
0.0020562744029710704j [Y0 X1 Y4 Y5] +
-0.005083283191416048j [Y0 X1 X6 X7] +
0.005083283191416048j [Y0 X1 Y6 Y7] +
-0.006478936130102838j [Y0 Y1 X2 Y3] +
-0.006478936130102838j [Y0 Y1 Y2 X3] +
-0.0020562744029710704j [Y0 Y1 X4 Y5] +
-0.0020562744029710704j [Y0 Y1 Y4 X5] +
-0.0

In [15]:
num_electrons = calculated_molecule.n_electrons
num_orbitals = calculated_molecule.n_orbitals
print(num_electrons, num_orbitals)

2 4


In [27]:
for index, i in enumerate(jw_configuration_state([0, 2, 4, 6], 8)):
    if i == 1:
        print(bit.int_to_bin(index, num_qubits))

10101010


In [3]:
hamiltonian = qonvert.QubitOperator_to_dict(ham_q, num_qubits)
vqe_input_ham = qonvert.dict_to_WeightedPauliOperator(hamiltonian)

In [40]:
hamiltonian_ref = deepcopy(qonvert.QubitOperator_to_dict(ham_q, num_qubits))
hamiltonian = qonvert.QubitOperator_to_dict(ham_q, num_qubits)
for i in range(20):
    terms_noncon = c_tools.greedy_dfs(hamiltonian, 3, criterion='weight')[-1]
    ham_noncon = {p:hamiltonian_ref[p] for p in terms_noncon}
    print(c_tools.find_gs_noncon(ham_noncon)[0])
    hamiltonian = {p:hamiltonian_ref[p] for p in hamiltonian.keys() if p not in ham_noncon.keys()}
    print(len(hamiltonian))

-3.1428247492681844
310
-0.7054115673880523
266
-0.35689014638003114
244
-0.3393570837487566
225
-0.26104466810786875
208
-0.2740224135405467
191
-0.21167184967401675
174
-0.2039615888273435
147
-0.18195728287942495
130
-0.1533751499176475
103
-0.12264307368181022
89
-0.08759091716794166
76
-0.06371990002241652
66
-0.08329647710954567
52
-0.046211359286214164
40
-0.04546562337603891
30
-0.032235625229371956
23
-0.024442898703625454
16
-0.010404316549836092
10
-0.00566490129361918
5


In [33]:
hamiltonian_ref

{'IIIIIIII': (1.1324434021069383+0j),
 'ZIIIIIII': (0.1399063176082609+0j),
 'YZYIIIII': (0.002577905179419724+0j),
 'XZXIIIII': (0.002577905179419724+0j),
 'YZZZYIII': (0.057762059101369584+0j),
 'XZZZXIII': (0.057762059101369584+0j),
 'YZZZZZYI': (-0.0386607066686353+0j),
 'XZZZZZXI': (-0.0386607066686353+0j),
 'IZIIIIII': (0.13990631760826078+0j),
 'IYZYIIII': (0.0025779051794197093+0j),
 'IXZXIIII': (0.0025779051794197093+0j),
 'IYZZZYII': (0.0577620591013696+0j),
 'IXZZZXII': (0.0577620591013696+0j),
 'IYZZZZZY': (-0.038660706668635275+0j),
 'IXZZZZZX': (-0.038660706668635275+0j),
 'IIZIIIII': (-0.24607368520020115+0j),
 'IIYZYIII': (0.061555459243441205+0j),
 'IIXZXIII': (0.061555459243441205+0j),
 'IIYZZZYI': (0.020218321802530737+0j),
 'IIXZZZXI': (0.020218321802530737+0j),
 'IIIZIIII': (-0.24607368520020115+0j),
 'IIIYZYII': (0.06155545924344121+0j),
 'IIIXZXII': (0.06155545924344121+0j),
 'IIIYZZZY': (0.020218321802530734+0j),
 'IIIXZZZX': (0.020218321802530734+0j),
 'IIIIZII

In [5]:
circs = cs_circ.cs_vqe_circuit(hamiltonian, terms_noncon, num_qubits, num_electrons, rot_A=True)

In [13]:
circs.run_cs_vqe(ucc, max_sim_q = 4, noise=True)

*Performing 1-qubit CS-VQE over qubit positions 7
Expectation value of A: 0.806640625


/home/tim/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1268: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)


 
*Performing 2-qubit CS-VQE over qubit positions 7, 1
Expectation value of A: 0.916015625
 
*Performing 3-qubit CS-VQE over qubit positions 7, 1, 0
Expectation value of A: 0.943359375


KeyboardInterrupt: 

In [ ]:
cs_vqe_results = circs.cs_vqe_results

In [ ]:
fig = plot.plot_cs_vqe_convergence(data=cs_vqe_results, title='test')

In [ ]:
#import json 

#data={}
#for k in cs_vqe_results.keys():
#    data[str(k)] = cs_vqe_results[k]
     
#with open("data/BeH-_singlet_UCCSD_HF_CS-VQE_convergence_with_ground_state_projection_SUPPLEMENT.json", "w") as outfile: 
#    json.dump(data, outfile)

In [ ]:
circs.gs_noncon_energy

In [ ]:
mol = c.cs_vqe(hamiltonian, terms_noncon, num_qubits)
generators = list((mol.generators()[0]).keys())
initial_state = mol.init_state()
results={}
shots=10
c_order = [0,1,2,3,4,5]

eig_proj = la.eigenstate_projector(A, num_qubits)
nan_check = []

for index in range(num_qubits):
    removed_index = c_order[0:index]#list(range(index))
    removed_index.reverse()
    removed_generators = [generators[i] for i in removed_index]
    Z_indices = [g.find('Z') for g in removed_generators]
    print(removed_generators, Z_indices)
    nc_proj = la.noncon_projector(initial_state, Z_indices, num_qubits)

    new_ham_noncon, new_ham_context = mol.move_generator(removed_generators)
    new_ham_noncon_q = qonvert.dict_to_QubitOperator(new_ham_noncon)
    new_ham_context_q = qonvert.dict_to_QubitOperator(new_ham_context)

    results[index] = {}
    results[index]['exact'] = get_ground_state(get_sparse_operator(new_ham_noncon_q, num_qubits).toarray())[0]

    expectations = []
    for r in range(shots):
        rand_vec = la.random_complex_vector(2**num_qubits)
        psi = la.apply_projections(rand_vec, [nc_proj, eig_proj])
        expect_noncon = la.expectation(new_ham_noncon_q, psi, num_qubits)
        expectations.append(expect_noncon)
    #print(expectations)
    nc_value = (sum(expectations)/shots).real
    results[index]['+1eig'] = nc_value
    nan_check.append(nc_value)

results

In [ ]:
X=range(num_qubits)
values = list(zip(*[list(results[i].values()) for i in X]))
print(values)
Y1=values[0]
Y2=values[1]
#axs[grid].set_title(str(h))
plt.plot(X, Y1, label='Exact noncontextual gs')
plt.plot(X, Y2, label='Restriction to +1-eigenspace')
plt.legend()

In [ ]:
#fig.savefig("plots/HeH+-CS-VQE-uccsd_ansatz.png", dpi=300)

In [ ]:
circs.ancilla = False
X = list(range(1, 6))
Y_true = []
Y_proj = []
A = circs.A

for n_q in X:
    ham_red = circs.ham_reduced[n_q-1]
    ham_red_q = qonvert.dict_to_QubitOperator(ham_red)
    gs_red = get_ground_state(get_sparse_operator(ham_red_q, n_q).toarray())
    Y_true.append(gs_red[0])
    
    A_red = circs.reduce_anz_terms(A, n_q)
    eig_proj = la.eigenstate_projector(A_red, n_q)
    psi = gs_red[1]
    psi_proj = la.apply_projections(psi, [eig_proj])
    gs_proj = la.expectation(ham_red, psi_proj, n_q)
    Y_proj.append(gs_proj)
    
plt.plot(X, Y_true, label = 'True ground state energy')
plt.plot(X, Y_proj, label = 'Expectation value after projection')
plt.legend()

In [ ]:
rots = circs.all_rotations
print(rots)
rots.reverse()
c_tools.rotate_operator(rots, A)

In [ ]:
circs.G

In [ ]:
P1, P2 = A.keys()
r1 = A[P1]
r2 = A[P2]
rot = c_tools.pauli_mult(P1, P2)
t = np.arctan(r1/r2)
single_A = c_tools.rotate_operator([[t*(rot[1]*1j).real, rot[0]]], A)

In [ ]:
Z_indices = [g.find('Z') for g in circs.G]

In [ ]:
single_A_indices = [i for i, p in enumerate(list(single_A.keys())[0]) if p == 'Z']

In [ ]:
ind_Z = [i for i in single_A_indices if i not in Z_indices]

In [ ]:
ind_Z

In [ ]:
diag_A = []
for i in ind_Z:    
    blank_op = ['I' for i in range(num_qubits)]
    blank_op[i] = 'Y'
    rot = ''.join(blank_op)
    diag_A.append(['pi/2', rot])
    rotated = c_tools.pauli_mult(rot, list(single_A.keys())[0])[0]
    print(rotated)
    blank_op = ['I' for i in range(num_qubits)]
    for j in single_A_indices:
        if rotated[j]=='Z':
            blank_op[j] = 'Z'
        elif rotated[j] == 'X':
            blank_op[j] = 'Y'
    rot = ''.join(blank_op)
    diag_A.append(['pi/2', rot])

In [ ]:
c_tools.pauli_mult(rot_G, list(single_A.keys())[0])

In [ ]:
A_proj = np.matrix(la.eigenstate_projector(circs.A, num_qubits))
A_proj.trace()

In [ ]:
from qiskit.circuit import ParameterVector

In [ ]:
list(ParameterVector('P', 2))

In [ ]:
circuit = circs.build_circuit(ucc, 2)
circuit.draw()

In [ ]:
circuit.

In [ ]:
list(circs.build_circuit(ucc, 3).parameters)

In [15]:
from collections import Counter
atoms = ['H', 'H', 'O']
mult = Counter(atoms)

In [24]:
speciesname

'H2_O1_sto-3g1.342396'